In [1]:
import pandas as pd
import streamlit as st
import pandas as pd
from unidecode import unidecode
import re
import numpy as np

In [9]:
def xu_ly_file_upload(uploaded_file, loai_bao_cao, file_extension='xlsx'):
    import pandas as pd

    if uploaded_file is None:
        return None

    # Đọc file không có header để dò dòng tiêu đề
    if file_extension in ['xlsx', 'xls']:
        df = pd.read_excel(uploaded_file, header=None)
    elif file_extension == 'csv':
        df = pd.read_csv(uploaded_file, header=None, encoding='utf-8')
    else:
        return None

    current_year = pd.Timestamp.now().year
    start_row = None

    for i in range(len(df)):
        row = df.iloc[i]
        year_count = 0

        # Bỏ qua dòng nếu cột đầu tiên là năm
        try:
            first_val = float(row[0])
            if first_val.is_integer() and 2000 <= int(first_val) <= current_year:
                continue  # Không hợp lệ nếu cột đầu là năm
        except:
            pass  # OK nếu không phải năm

        # Đếm số cột là năm (từ cột thứ 2 trở đi)
        for j in range(1, len(row)):
            try:
                val = float(row[j])
                if val.is_integer() and 2000 <= int(val) <= current_year:
                    year_count += 1
            except:
                continue

        # Chấp nhận nếu có ít nhất 3 cột là năm
        if year_count >= 3:
            start_row = i
            break

    if start_row is None:
        return None

    # Đọc lại file với header đúng
    if file_extension in ['xlsx', 'xls']:
        df = pd.read_excel(uploaded_file, header=start_row)
    elif file_extension == 'csv':
        df = pd.read_csv(uploaded_file, header=start_row, encoding='utf-8')

    df.columns = df.columns.astype(str).str.strip()
    df.fillna(0, inplace=True)

    return df


In [3]:
search_terms_LCTT = {
    # LCTT (Báo cáo Lưu chuyển Tiền tệ)
    "DTRTHDKD": "Lưu chuyển tiền thuần từ hoạt động kinh doanh",
    "DTRTHDDT": "Lưu chuyển tiền thuần từ hoạt động đầu tư",
    "DTRTHDTC": "Lưu chuyển tiền thuần từ hoạt động tài chính",
    "KH": "Khấu hao TSCĐ và BĐSĐT",
}

search_terms_CDKT = {
    # CDKT (Bảng Cân đối Kế toán)
    "TTS": "TỔNG CỘNG TÀI SẢN",
    "TSNH": "TÀI SẢN NGẮN HẠN",
    "TVCKTDT": "Tiền và các khoản tương đương tiền",
    "HTK": "Hàng tồn kho",
    "TSCDHH": "Tài sản cố định hữu hình",
    "TSCD": "Tài sản cố định",
    "CKPTNH": "Các khoản phải thu ngắn hạn",
    "VCSH": "VỐN CHỦ SỞ HỮU",
    "NPT": "NỢ PHẢI TRẢ",
    "NNH": "Nợ ngắn hạn",
    "NDH": "Nợ dài hạn",
    "PTNBNH": "Phải trả người bán ngắn hạn",
}

search_terms_KQKD = {
    # KQHDKD (Báo cáo Kết quả Hoạt động Kinh doanh)
    "DTT": "Doanh thu thuần về bán hàng và cung cấp dịch vụ",
    "LNT": "Lợi nhuận thuần từ hoạt động kinh doanh",
    "LNR": "Lợi nhuận sau thuế thu nhập doanh nghiệp",
    "LNG": "Lợi nhuận gộp về bán hàng và cung cấp dịch vụ",
    "GVHB": "Giá vốn hàng bán",
    "LCBTCP": "Lãi cơ bản trên cổ phiếu",
    "CPLV": "Chi phí lãi vay",
}

In [4]:
def normalize_text(text):
    # Kiểm tra nếu text là kiểu chuỗi
    if isinstance(text, str):
        # Chuyển thành chữ thường và loại bỏ dấu tiếng Việt
        text = text.lower()
        text = unidecode(text)
        text = text.strip()
        # Loại bỏ các ký tự không mong muốn (ngoài a-z, 0-9 và khoảng trắng)
        text = re.sub(r'[^a-z0-9\s]', '', text)
    # Nếu là kiểu số thì không làm gì, giữ nguyên
    return text

def clean_column_name(col):
    try:
        num = float(col)
        if num.is_integer():
            return str(int(num))  # Bỏ '.0'
        else:
            return str(num)
    except:
        return col  # Nếu không chuyển được thì giữ nguyên

In [7]:
def gop_file(df_CDKT, df_KQKD, df_LCTT, Ma_Cty):
    # Chuẩn hóa tên cột
    for df in [df_CDKT, df_KQKD, df_LCTT]:
        df.columns = df.columns.astype(str).str.strip()
         
    df_CDKT.fillna(0, inplace=True)
    df_KQKD.fillna(0, inplace=True)
    df_LCTT.fillna(0, inplace=True) 
    
    # Dictionary chứa dữ liệu cho từng công ty
    df_total = pd.DataFrame([])
    dong_hien_tai = 0
    
    #Lấy năm của công ty
    Nam_CDKT = df_CDKT.columns.tolist()
    Nam_KQKD = df_KQKD.columns.tolist()
    Nam_LCTT = df_LCTT.columns.tolist()

    # Chuyển các danh sách thành tập hợp (set) và lấy giao (intersection)
    Nam_trung = set(Nam_CDKT) & set(Nam_KQKD) & set(Nam_LCTT)

    # Xóa số 0, sắp xếp tăng dần
    Nam_trung = sorted(n for n in Nam_trung if n != '0' and n != 'Chỉ số')
    # Dictionary chứa dữ liệu cho từng công ty

    for year in Nam_trung:  # Lặp qua từng năm trùng
        df_total.at[dong_hien_tai, 'Id'] = int(dong_hien_tai + 1)
        df_total.at[dong_hien_tai, 'Ma_Cty'] = Ma_Cty
        df_total.at[dong_hien_tai, 'Nam'] = year

        # Áp dụng normalize_text vào cột đầu tiên của DataFrame
        df_CDKT.iloc[:, 0] = df_CDKT.iloc[:, 0].apply(normalize_text)
        df_KQKD.iloc[:, 0] = df_KQKD.iloc[:, 0].apply(normalize_text)
        df_LCTT.iloc[:, 0] = df_LCTT.iloc[:, 0].apply(normalize_text)

        # Duyệt qua các từ khóa tìm kiếm
        for key, term in search_terms_CDKT.items():  
            df_processing = df_CDKT              
            # Tìm trong các file, sử dụng normalize_text cho cột đầu tiên\
            matched_rows = df_processing.loc[df_processing.iloc[:, 0].str.contains(normalize_text(term), na=False), year]
           
            value = matched_rows.values

            if value.size >= 1:
                df_total.at[dong_hien_tai, key] = float(value[0])

        # Duyệt qua các từ khóa tìm kiếm
        for key, term in search_terms_KQKD.items():  
            df_processing = df_KQKD              
            # Tìm trong các file, sử dụng normalize_text cho cột đầu tiên\
            matched_rows = df_processing.loc[df_processing.iloc[:, 0].str.contains(normalize_text(term), na=False), year]
            value = matched_rows.values
            
            if value.size >= 1:
                df_total.at[dong_hien_tai, key] = float(value[0])      
                
        for key, term in search_terms_LCTT.items():  
            df_processing = df_LCTT              
            # Tìm trong các file, sử dụng normalize_text cho cột đầu tiên\
            matched_rows = df_processing.loc[df_processing.iloc[:, 0].str.contains(normalize_text(term), na=False), year]
            value = matched_rows.values
            
            if value.size >= 1:
                df_total.at[dong_hien_tai, key] = float(value[0])
        
        dong_hien_tai += 1  # Tiến sang dòng tiếp theo
        

    # Lấy index của các dòng có NaN trong 'Nam'
    na_indexes = df_total[df_total['Nam'].isna()].index
    print("Chỉ số các dòng có NaN trong cột 'Nam':", list(na_indexes))

    # Xoá các dòng có NaN trong 'Nam'
    df_total = df_total.drop(index=na_indexes).reset_index(drop=True)
    
    # Đổi kiểu dữ liệu
    for column in df_total.columns[3:]:
        df_total[column] = pd.to_numeric(df_total[column], errors='coerce').astype('float64')
        
    df_total['Nam'] = pd.to_numeric(df_total['Nam'], errors='coerce').astype('Int64')

    # Gộp tất cả các từ khóa lại với hậu tố tương ứng
    all_keys = (
        [key for key in search_terms_LCTT.keys()] +
        [key for key in search_terms_CDKT.keys()] +
        [key for key in search_terms_KQKD.keys()]
    )

    # Lấy danh sách các cột trong dataframe
    df_columns = df_total.columns.tolist()

    # Tìm các key bị thiếu
    missing_keys = [key for key in all_keys if key not in df_columns]

    # In ra kết quả
    if not missing_keys:
        print("✅ Tất cả các key đều có mặt trong df.columns")
    else:
        print("❌ Thiếu các key sau trong df.columns:")
        print(missing_keys)
    return df_total

In [10]:
df_CDKT = xu_ly_file_upload(r"D:\Nam4\KHÓA LUẬN\DATA TEST\AAA_CDKT.xlsx", 'CDKT')
df_LCTT = xu_ly_file_upload(r"D:\Nam4\KHÓA LUẬN\DATA TEST\AAA_LCTT.xlsx", 'LCTT')
df_KQKD = xu_ly_file_upload(r"D:\Nam4\KHÓA LUẬN\DATA TEST\AAA_KQKD.xlsx", 'KQKD')

In [11]:
gop_file(df_CDKT, df_KQKD, df_LCTT, 'AAA')

C:\Users\Kim Ly\AppData\Local\Temp\ipykernel_11412\501029274.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AAA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_total.at[dong_hien_tai, 'Ma_Cty'] = Ma_Cty
C:\Users\Kim Ly\AppData\Local\Temp\ipykernel_11412\501029274.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2007' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_total.at[dong_hien_tai, 'Nam'] = year


Chỉ số các dòng có NaN trong cột 'Nam': []
✅ Tất cả các key đều có mặt trong df.columns


,Id,Ma_Cty,Nam,TTS,TSNH,TVCKTDT,HTK,TSCDHH,TSCD,CKPTNH,...,LNT,LNR,LNG,GVHB,LCBTCP,CPLV,DTRTHDKD,DTRTHDDT,DTRTHDTC,KH
0,1.0,AAA,2007,1.535104e+08,1.002769e+08,8.973523e+06,3.527572e+07,4.352466e+07,5.308765e+07,4.268377e+07,...,16298742.0,16407162.0,2.264518e+07,8.589211e+07,2835.0,1109102.0,-4.993301e+07,-6.458597e+07,1.234925e+08,3327113.0
1,2.0,AAA,2008,2.713314e+08,1.435537e+08,3.681471e+06,3.997763e+07,1.022641e+08,1.259990e+08,9.009276e+07,...,22821680.0,22874647.0,5.197842e+07,1.963550e+08,3445.0,13158802.0,-4.637282e+07,-6.124157e+07,1.021796e+08,6302616.0
2,3.0,AAA,2009,4.358508e+08,1.828263e+08,3.782176e+07,4.869050e+07,2.023987e+08,2.102785e+08,8.457640e+07,...,37817350.0,37722752.0,8.359756e+07,3.009359e+08,5716.0,17140936.0,5.232961e+07,-1.146798e+08,9.644083e+07,12603828.0
3,4.0,AAA,2010,6.449785e+08,2.846300e+08,2.019609e+07,8.061924e+07,3.016047e+08,3.440407e+08,1.619500e+08,...,98906274.0,90222093.0,1.866899e+08,4.926248e+08,10315.0,23323421.0,-7.977260e+05,-1.607052e+08,1.038720e+08,25694184.0
4,5.0,AAA,2011,8.166180e+08,2.946820e+08,3.292108e+07,1.255990e+08,4.151586e+08,4.387731e+08,1.104259e+08,...,76689465.0,65671439.0,1.977429e+08,7.128912e+08,6614.0,34840627.0,5.613286e+07,-1.641219e+08,1.207046e+08,34064170.0
5,6.0,AAA,2012,9.004938e+08,4.330031e+08,1.034210e+08,1.955777e+08,3.828795e+08,4.305834e+08,9.381282e+07,...,64021126.0,56177588.0,1.863395e+08,8.236937e+08,4850.0,29362607.0,1.111205e+08,-1.580920e+07,-2.479294e+07,46632888.0
6,7.0,AAA,2013,1.150569e+09,4.950250e+08,1.302667e+08,2.000260e+08,5.341738e+08,6.051598e+08,1.469424e+08,...,63819232.0,55471541.0,1.785352e+08,9.789726e+08,2775.0,17108189.0,8.352444e+07,-2.086415e+08,1.518161e+08,48746209.0
7,8.0,AAA,2014,1.421651e+09,6.943792e+08,3.368077e+08,1.375221e+08,6.183029e+08,6.799688e+08,1.540885e+08,...,50265261.0,47780875.0,1.827449e+08,1.377899e+09,1895.0,17187220.0,9.929954e+07,-1.275599e+08,2.344224e+08,64200063.0
8,9.0,AAA,2015,1.954765e+09,1.071561e+09,4.700617e+08,2.141866e+08,5.953754e+08,6.365322e+08,3.635097e+08,...,48754415.0,40548814.0,1.898078e+08,1.424741e+09,642.0,24065811.0,-7.334908e+07,-2.443692e+08,4.508355e+08,83358285.0
9,10.0,AAA,2016,3.077616e+09,1.361646e+09,4.066768e+08,4.505913e+08,1.338478e+09,1.434278e+09,4.555676e+08,...,165553233.0,142925018.0,3.070354e+08,1.836734e+09,2611.0,27284472.0,8.016955e+07,-1.011689e+09,8.680370e+08,92958744.0
